In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import optuna

In [2]:
df = pd.read_csv(r'C:\Users\Professional\Desktop\Kaggle competitions\30-days-of-ml\data\train_folds.csv')
df_test = pd.read_csv(r'C:\Users\Professional\Desktop\Kaggle competitions\30-days-of-ml\data\test.csv')
sample_submission = pd.read_csv(r'C:\Users\Professional\Desktop\Kaggle competitions\30-days-of-ml\data\sample_submission.csv')

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if col.startswith("cat")]
df_test = df_test[useful_features]

for col in object_cols:
    temp_df = []
    temp_test_feat = None
    for fold in range(5):
        xtrain =  df[df.kfold != fold].reset_index(drop=True)
        xvalid = df[df.kfold == fold].reset_index(drop=True)
        feat = xtrain.groupby(col)["target"].agg("mean")
        feat = feat.to_dict()
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat = df_test[col].map(feat)
        else:
            temp_test_feat += df_test[col].map(feat)
    
    temp_test_feat /= 5
    df_test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    df = pd.concat(temp_df)
    

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if col.startswith("cat")]
df_test = df_test[useful_features]

In [26]:
def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.4, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 8)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = CatBoostRegressor(
        random_state=42,
        task_type = "GPU",
        n_estimators=8000,
        bootstrap_type="Bernoulli",
        learning_rate=learning_rate,
        subsample=subsample,
        reg_lambda=reg_lambda,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=800, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [27]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=15)

[I 2021-08-24 07:38:24,381] A new study created in memory with name: no-name-e4d5f4ab-d4a5-4a12-a9ef-bd8a85121178


0:	learn: 0.7463312	test: 0.7452627	best: 0.7452627 (0)	total: 7.09ms	remaining: 56.7s
1000:	learn: 0.7178207	test: 0.7230532	best: 0.7230527 (999)	total: 6.41s	remaining: 44.8s
2000:	learn: 0.7094455	test: 0.7213875	best: 0.7213875 (2000)	total: 12.9s	remaining: 38.7s
3000:	learn: 0.7020173	test: 0.7208884	best: 0.7208817 (2988)	total: 19.4s	remaining: 32.2s
4000:	learn: 0.6951390	test: 0.7207361	best: 0.7207326 (3994)	total: 25.7s	remaining: 25.7s
bestTest = 0.7207211196
bestIteration = 4083
Shrink model to first 4084 iterations.


[I 2021-08-24 07:38:58,469] Trial 0 finished with value: 0.7207209795491829 and parameters: {'learning_rate': 0.033283368472661196, 'reg_lambda': 6.180730194339443, 'subsample': 0.2093815583335964, 'colsample_bylevel': 0.5198618986889302, 'max_depth': 6}. Best is trial 0 with value: 0.7207209795491829.


0:	learn: 0.7467278	test: 0.7456590	best: 0.7456590 (0)	total: 5.75ms	remaining: 46s
1000:	learn: 0.7355175	test: 0.7349490	best: 0.7349490 (1000)	total: 5.12s	remaining: 35.8s
2000:	learn: 0.7319088	test: 0.7317241	best: 0.7317241 (2000)	total: 10.3s	remaining: 30.7s
3000:	learn: 0.7297008	test: 0.7298690	best: 0.7298690 (3000)	total: 15.4s	remaining: 25.7s
4000:	learn: 0.7279163	test: 0.7284232	best: 0.7284232 (4000)	total: 20.5s	remaining: 20.5s
5000:	learn: 0.7265135	test: 0.7273825	best: 0.7273825 (5000)	total: 25.6s	remaining: 15.3s
6000:	learn: 0.7252472	test: 0.7264730	best: 0.7264730 (6000)	total: 30.7s	remaining: 10.2s
7000:	learn: 0.7241604	test: 0.7257569	best: 0.7257569 (7000)	total: 35.7s	remaining: 5.1s
7999:	learn: 0.7232349	test: 0.7251442	best: 0.7251442 (7999)	total: 40.8s	remaining: 0us
bestTest = 0.7251441577
bestIteration = 7999


[I 2021-08-24 07:39:42,161] Trial 1 finished with value: 0.7251442280314992 and parameters: {'learning_rate': 0.003738944269756206, 'reg_lambda': 0.2302525692496485, 'subsample': 0.7783662964379253, 'colsample_bylevel': 0.9617592349898778, 'max_depth': 4}. Best is trial 0 with value: 0.7207209795491829.


...

[I 2021-08-24 07:44:31,935] Trial 12 finished with value: 0.7222602809068469 and parameters: {'learning_rate': 0.09214008867048275, 'reg_lambda': 0.28119876152359363, 'subsample': 0.25709022568392054, 'colsample_bylevel': 0.4970640998972675, 'max_depth': 8}. Best is trial 8 with value: 0.7206880949738793.


0:	learn: 0.7435116	test: 0.7424494	best: 0.7424494 (0)	total: 6.88ms	remaining: 55s
1000:	learn: 0.6648464	test: 0.7269283	best: 0.7223076 (231)	total: 6.33s	remaining: 44.2s
bestTest = 0.722307613
bestIteration = 231
Shrink model to first 232 iterations.


[I 2021-08-24 07:44:40,794] Trial 13 finished with value: 0.7223076033353495 and parameters: {'learning_rate': 0.2792867688001318, 'reg_lambda': 2.6459690383287824, 'subsample': 0.2542996145989843, 'colsample_bylevel': 0.11804148543646666, 'max_depth': 6}. Best is trial 8 with value: 0.7206880949738793.


0:	learn: 0.7465828	test: 0.7455174	best: 0.7455174 (0)	total: 4.46ms	remaining: 35.7s


KeyboardInterrupt: 

In [5]:
study.best_params

{'learning_rate': 0.019620560209436984,
 'reg_lambda': 0.01011649689512418,
 'subsample': 0.5976074901918561,
 'colsample_bylevel': 0.3858760189885154,
 'max_depth': 6}